In [1]:
!pip install streamlit-folium
!pip install geopandas

     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 8.3 MB 16.0 MB/s 
     |████████████████████████████████| 111 kB 58.8 MB/s 
     |████████████████████████████████| 180 kB 53.8 MB/s 
     |████████████████████████████████| 4.3 MB 30.1 MB/s 
     |████████████████████████████████| 76 kB 4.8 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 124 kB 51.8 MB/s 
     |████████████████████████████████| 788 kB 52.4 MB/s 
     |████████████████████████████████| 370 kB 75.9 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=dc84c1b13b9079e0c2b036e9e9cb43dc621ddf01f0a39e3e708864ea41f32522
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Su

     |████████████████████████████████| 1.0 MB 33.0 MB/s 
     |████████████████████████████████| 15.4 MB 27 kB/s 
     |████████████████████████████████| 6.3 MB 39.1 MB/s 


In [2]:
import zipfile
import requests
import pandas as pd
import numpy as np
import xarray as xr
import shapely
import geopandas as gpd
import folium

url = 'https://africaopendata.org/dataset/83582021-d8f7-4bfd-928f-e9c6c8cb1247/resource/372a616a-66cc-41f7-ac91-d8af8f23bc2b/download/nigeria-lgas.zip'

r = requests.get(url)

with open('nigeria.zip', 'wb') as f:
  f.write(r.content)

with zipfile.ZipFile('nigeria.zip') as myzip:
  myzip.extractall('/content/')


url = 'https://climatedata.worldbank.org/thredds/fileServer/CRM/cru/cru-ts4.05-climatology/tas/mean/monthly/climatology-tas-monthly-mean/1991-2020/climatology-tas-monthly-mean_cru_monthly_cru-ts4.05-climatology_mean_1991-2020.nc'

r = requests.get(url)

with open('temperature.nc', 'wb') as f:
  f.write(r.content)


url = 'https://climatedata.worldbank.org/thredds/fileServer/CRM/cru/cru-ts4.05-climatology/pr/mean/monthly/climatology-pr-monthly-mean/1991-2020/climatology-pr-monthly-mean_cru_monthly_cru-ts4.05-climatology_mean_1991-2020.nc'

r = requests.get(url)

with open('precipitation.nc', 'wb') as f:
  f.write(r.content)

In [68]:
%%writefile app.py

import streamlit as st
import pandas as pd
import xarray as xr
import folium
from streamlit_folium import folium_static
import geopandas as gpd
import shapely
import time



nigeria_shp = gpd.read_file('/content/new_lga_nigeria_2003.shp')
bounds = nigeria_shp.dissolve().bounds

precip = xr.open_dataset('/content/precipitation.nc')

precip = precip.loc[dict(lat = precip['lat'].where((precip.lat >= bounds.miny[0]) & (precip.lat <= bounds.maxy[0] ), drop = True), 
                         lon =precip['lon'].where((precip.lon>= bounds.minx[0]) & (precip.lon<= bounds.maxx[0] ), drop=True))]

temp = xr.open_dataset('/content/temperature.nc')

temp = temp.loc[dict(lat = temp['lat'].where((temp.lat >= bounds.miny[0]) & (temp.lat <= bounds.maxy[0] ), drop = True), 
                         lon =temp['lon'].where((temp.lon>= bounds.minx[0]) & (temp.lon<= bounds.maxx[0] ), drop=True))]

temp_pr = temp.to_dataframe().dropna().reset_index().loc[:,['lat', 'lon','time', 'climatology-tas-monthly-mean']].drop_duplicates().merge(precip.to_dataframe().dropna().reset_index().loc[:,['lat', 'lon','time', 'climatology-pr-monthly-mean']].drop_duplicates(), on = ['lat','lon','time']).drop_duplicates()

temp_pr.reset_index(inplace=True)
temp_pr.columns = ['division', 'lat', 'lon', 'time', 'temp', 'precip']

gdf = temp_pr.copy()

gdf = gpd.GeoDataFrame(gdf, geometry=gdf.apply(lambda x: shapely.geometry.box(x['lon']-0.25, x['lat']-0.25, x['lon']+0.25, x['lat']+0.25 ), axis=1))
gdf.set_crs(epsg=4326, inplace=True)
gdf['Month_name'] = gdf['time'].dt.strftime('%b')

x = nigeria_shp.dissolve().to_crs(epsg=4087).centroid.to_crs(4326)[0].x
y = nigeria_shp.dissolve().to_crs(epsg=4087).centroid.to_crs(4326)[0].y


option_dict = {'Temperature':'temp', 'Rainfall':'precip'}

map_dict = {}

for k,v in option_dict.items():
  if v == 'temp':
    color = 'OrRd'
  else:
    color = 'Blues'
  
  month_dict = {}
  for month in gdf['Month_name'].unique():
    m = folium.Map(location=[y,x], zoom_start=6)


    choropleth = folium.Choropleth(
        geo_data=gdf.loc[gdf['Month_name']==month].reset_index()[['division', v,'geometry']],
        name="choropleth",
        data=gdf.loc[gdf['Month_name']==month].reset_index()[['division', v,'geometry']],
        columns=['division',v],
        key_on="feature.properties.division",
        fill_color=color,
        fill_opacity=0.7,
        line_opacity=0.1,
        legend_name=v,
    ).add_to(m)

    choropleth.geojson.add_child(
        folium.features.GeoJsonTooltip([v])
    )

    month_dict[month] = m
  map_dict[k] = month_dict



def map_gen(variable):
  while True:
    for month in gdf['Month_name'].unique():
      yield month, map_dict[variable][month]


selected = st.selectbox('Pick a variable', options=option_dict.keys())
placeholder = st.empty()
gen_obj = map_gen(selected)
while True:
  month, m = next(gen_obj)
  with placeholder.container():
    st.write(selected + ' for the month of '+ month)
    folium_static(m)
    time.sleep(5)


Overwriting app.py


In [47]:
!streamlit run app.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.86.94.24:8501

  Stopping...


In [8]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2021-10-19 04:25:52--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  79.1MB/s    in 0.2s    

2021-10-19 04:25:52 (79.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]



In [9]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [69]:
get_ipython().system_raw('./ngrok http 8501 &')

In [70]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://9305-34-86-94-24.ngrok.io


In [71]:
!streamlit run app.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.86.94.24:8501

2021-10-19 05:26:34.078 NumExpr defaulting to 2 threads.
  Stopping...
  Stopping...
